# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score


### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [ ]:
# Dicionário para mapear os IDs das atividades para os nomes corretos
activity_labels = {
    1: "Walking",
    2: "Walking Upstairs",
    3: "Walking Downstairs",
    4: "Sitting",
    5: "Standing",
    6: "Laying"
}

def process_data(caminho_features, caminho_subject, caminho_X, caminho_y, tipo='train'):
    # Criar diretório de output se não existir
    os.makedirs("./output", exist_ok=True)

    # Carregar features.txt em uma Series (nomes das colunas)
    features_df = pd.read_csv(caminho_features, delim_whitespace=True, header=None, usecols=[1])
    features = features_df.squeeze().str.strip()  # Remover espaços extras
    features.name = 'Var'

    # Carregar subject_train.txt ou subject_test.txt (IDs dos participantes)
    subject_df = pd.read_csv(caminho_subject, header=None, names=['subject'])
    subject = subject_df.squeeze()

    # Carregar y_train.txt ou y_test.txt (variável resposta)
    y = pd.read_csv(caminho_y, delim_whitespace=True, header=None, names=['Activity'])

    # Mapear os rótulos numéricos para nomes de atividades
    y['Activity'] = y['Activity'].map(activity_labels)

    # Carregar X_train.txt ou X_test.txt (dados dos sensores)
    X = pd.read_csv(caminho_X, delim_whitespace=True, header=None)

    # Renomear as colunas de X usando os nomes em features
    X.columns = features

    # Treinar um modelo para selecionar as 3 melhores variáveis
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X, y['Activity'])  
    importances = clf.feature_importances_
    
    # Selecionar as 3 melhores variáveis com base na importância
    top_features_idx = np.argsort(importances)[-3:]  # Índices das 3 mais importantes
    selected_columns = features.iloc[top_features_idx].values.tolist()  # Nomes das features

    # Criar DataFrame com as colunas selecionadas
    X_selected = X[selected_columns]

    # Salvar os DataFrames em arquivos CSV
    X.to_csv(f"./output/X_{tipo}.csv", index=False)
    y.to_csv(f"./output/y_{tipo}.csv", index=False)
    X_selected.to_csv(f"./output/X_{tipo}_selected.csv", index=False)

    return X, X_selected, y, selected_columns

# Aplicando a Validação Cruzada (K-Fold)
def cross_validate_model(X_train, y_train, ccp_alpha=0.0005, cv=5):
    model = DecisionTreeClassifier(ccp_alpha=ccp_alpha, random_state=42)
    
    # Aplicando Cross Validation
    scores = cross_val_score(model, X_train, y_train, cv=cv)
    
    print(f"Acurácias por fold: {scores}")
    print(f"Média da acurácia: {scores.mean():.4f}")
    
    return scores.mean()

# Processar os dados de treino e teste
X_train, X_train_selected, y_train, selected_features = process_data(
    "UCI HAR Dataset/features.txt",
    "UCI HAR Dataset/train/subject_train.txt",
    "UCI HAR Dataset/train/X_train.txt",
    "UCI HAR Dataset/train/y_train.txt",
    tipo="train"
)

X_test, X_test_selected, y_test, _ = process_data(
    "UCI HAR Dataset/features.txt",
    "UCI HAR Dataset/test/subject_test.txt",
    "UCI HAR Dataset/test/X_test.txt",
    "UCI HAR Dataset/test/y_test.txt",
    tipo="test"
)

# Rodar Validação Cruzada
best_accuracy = cross_validate_model(X_train_selected, y_train['Activity'], ccp_alpha=0.0005, cv=5)


C:\Users\djang\AppData\Local\Temp\ipykernel_7844\3211345365.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  features_df = pd.read_csv(caminho_features, delim_whitespace=True, header=None, usecols=[1])
C:\Users\djang\AppData\Local\Temp\ipykernel_7844\3211345365.py:25: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(caminho_y, delim_whitespace=True, header=None, names=['Activity'])
C:\Users\djang\AppData\Local\Temp\ipykernel_7844\3211345365.py:31: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X = pd.read_csv(caminho_X, delim_whitespace=True, header=None)


In [ ]:
%%time

# Código para treinar a árvore com o melhor ccp_alpha
from sklearn.tree import DecisionTreeClassifier

# Criar e treinar a árvore com o melhor ccp_alpha encontrado
melhor_ccp_alpha = 0.0005112202485292351  # Valor escolhido
arvore_final = DecisionTreeClassifier(ccp_alpha=melhor_ccp_alpha, random_state=42)
arvore_final.fit(X_train, y_train)

# Avaliação do modelo
acuracia_treino = arvore_final.score(X_train, y_train)
acuracia_teste = arvore_final.score(X_test, y_test)

print(f"Acurácia no treino: {acuracia_treino:.4f}")
print(f"Acurácia no teste: {acuracia_teste:.4f}")
